# Manual de Usuario - Biblioteca epsilon.py
## Herramienta de Post-Procesamiento para Tsunami-HySEA

---

**Versión:** 1.0  
**Fecha:** Febrero 2026  
**Desarrollado por:** EDANYA Research Group - Universidad de Málaga

---

## 📋 Tabla de Contenidos

1. [Introducción](#introduccion)
2. [Instalación y Requisitos](#requisitos)
3. [Inicio Rápido](#inicio-rapido)
4. [Funcionalidades Principales](#funcionalidades)
   - 4.1 [Selección de Simulaciones](#seleccion)
   - 4.2 [Visualización de Variables 2D](#visual-2d)
   - 4.3 [Visualización de Variables 3D (Temporales)](#visual-3d)
   - 4.4 [Perfiles Transversales](#transversales)
   - 4.5 [Series Temporales](#series-temporales)
   - 4.6 [Análisis de Inundación](#inundacion)
   - 4.7 [Tiempos de Llegada](#arrival-times)
   - 4.8 [Operaciones entre Variables](#operaciones)
5. [Exportación de Resultados](#exportacion)
6. [Ejemplos Prácticos](#ejemplos)
7. [Solución de Problemas](#problemas)
8. [Referencias](#referencias)

---
## 1. Introducción <a name="introduccion"></a>

La biblioteca **epsilon.py** es una herramienta de post-procesamiento diseñada para analizar y visualizar resultados de simulaciones de tsunamis generados por el software **Tsunami-HySEA**.

### Características principales:

- ✅ Interfaz gráfica interactiva con widgets de Jupyter
- ✅ Visualización de variables 2D y 3D (evolución temporal)
- ✅ Análisis de perfiles transversales
- ✅ Extracción de series temporales en puntos específicos
- ✅ Cálculo y visualización de zonas de inundación
- ✅ Mapas de tiempos de llegada del tsunami
- ✅ Operaciones algebraicas entre variables
- ✅ Exportación de figuras y animaciones GIF
- ✅ Soporte para coordenadas geográficas (lat/lon) y UTM

---
## 2. Instalación y Requisitos <a name="requisitos"></a>

### Requisitos del Sistema:

- Python 3.7 o superior
- Jupyter Notebook o JupyterLab
- Acceso a archivos NetCDF (.nc) de Tsunami-HySEA

### Dependencias de Python:

#### Opción 1: Instalación con conda (RECOMENDADA)

```bash
# Crear un entorno conda con todas las dependencias
conda create -n tsunami python=3.10
conda activate tsunami
conda install -c conda-forge numpy matplotlib ipywidgets netCDF4 xarray pyproj imageio gmt
pip install pygmt
```

#### Opción 2: Instalación con pip (sin PyGMT)

```bash
pip install numpy matplotlib ipywidgets netCDF4 xarray pyproj imageio
```

⚠️ **Nota sobre PyGMT**: PyGMT requiere que GMT (Generic Mapping Tools) esté instalado en el sistema. Si no tienes GMT instalado:
- **macOS**: `brew install gmt` o usar conda
- **Linux**: `sudo apt-get install gmt gmt-dcw gmt-gshhg` o usar conda
- **Alternativa**: epsilon.py funciona sin PyGMT, solo no mostrará mapas base con PyGMT

### Verificación de instalación:

In [1]:
# Verificar que todas las dependencias estén instaladas
import sys

# Paquetes esenciales (obligatorios)
essential_packages = [
    'numpy', 'matplotlib', 'ipywidgets', 
    'netCDF4', 'xarray', 'pyproj', 'imageio'
]

# Paquetes opcionales
optional_packages = ['pygmt']

print("=== PAQUETES ESENCIALES ===\n")
missing_essential = []
for package in essential_packages:
    try:
        __import__(package)
        print(f"✓ {package} instalado")
    except ImportError:
        print(f"✗ {package} NO instalado")
        missing_essential.append(package)

print("\n=== PAQUETES OPCIONALES ===\n")
for package in optional_packages:
    try:
        __import__(package)
        print(f"✓ {package} instalado y funcional")
    except ImportError:
        print(f"⚠️ {package} NO instalado (opcional - epsilon funcionará sin él)")
    except Exception as e:
        print(f"⚠️ {package} instalado pero con problemas: {type(e).__name__}")
        if package == 'pygmt':
            print("   → PyGMT requiere GMT instalado en el sistema")
            print("   → macOS: brew install gmt")
            print("   → Linux: sudo apt-get install gmt")
            print("   → O instalar con: conda install -c conda-forge gmt")
            print("   → epsilon.py funcionará pero sin mapas base PyGMT")

print("\n=== RESUMEN ===\n")
if missing_essential:
    print(f"❌ Faltan paquetes esenciales: {', '.join(missing_essential)}")
    print(f"   Instalar con: pip install {' '.join(missing_essential)}")
else:
    print("✅ Todos los paquetes esenciales están instalados")
    print("✅ epsilon.py está listo para usar")

=== PAQUETES ESENCIALES ===

✓ numpy instalado
✓ matplotlib instalado
✓ ipywidgets instalado
✓ netCDF4 instalado
✓ xarray instalado
✓ pyproj instalado
✓ imageio instalado

=== PAQUETES OPCIONALES ===

✓ pygmt instalado y funcional

=== RESUMEN ===

✅ Todos los paquetes esenciales están instalados
✅ epsilon.py está listo para usar


---
## 3. Inicio Rápido <a name="inicio-rapido"></a>

### Paso 1: Importar la biblioteca

**⚠️ IMPORTANTE**: Asegúrate de que el archivo `epsilon.py` esté en el mismo directorio que este notebook o en el PYTHONPATH.

**Verificación previa**: Antes de ejecutar la celda siguiente, verifica que:
1. El archivo `epsilon.py` existe en el directorio actual
2. El archivo `epsilon.py` es un archivo Python válido (no un notebook .ipynb renombrado)
3. Para verificar, puedes ejecutar: `head -5 epsilon.py` en terminal - debe mostrar código Python, no JSON

Si epsilon.py contiene JSON (como `{"cells": [...]`), significa que tienes un archivo notebook mal nombrado. Necesitas el archivo Python correcto.

In [ ]:
# Verificar que epsilon.py existe y es un archivo Python válido
import os

epsilon_path = os.path.join(os.getcwd(), 'epsilon.py')

if not os.path.exists(epsilon_path):
    print("❌ ERROR: epsilon.py no existe en el directorio actual")
    print(f"   Directorio actual: {os.getcwd()}")
    print(f"   Archivos disponibles: {[f for f in os.listdir() if f.endswith('.py')]}")
else:
    # Leer las primeras líneas para verificar que es código Python
    with open(epsilon_path, 'r') as f:
        first_lines = f.read(200)
    
    if first_lines.strip().startswith('{'):
        print("❌ ERROR: epsilon.py parece ser un archivo JSON/notebook, no un archivo Python")
        print("   Las primeras líneas son:")
        print(first_lines[:100])
        print("\n   Necesitas descargar el archivo epsilon.py correcto del repositorio")
    elif 'import' in first_lines or 'def' in first_lines:
        print("✅ epsilon.py existe y parece ser un archivo Python válido")
        print(f"   Ubicación: {epsilon_path}")
        print(f"   Tamaño: {os.path.getsize(epsilon_path)} bytes")
    else:
        print("⚠️ ADVERTENCIA: epsilon.py existe pero no se puede verificar su contenido")
        print(f"   Primeras líneas:\n{first_lines[:100]}")

In [ ]:
import sys
import os

# Obtiene la ruta absoluta del directorio actual de trabajo
current_dir = os.getcwd()

# Agregar el directorio actual al path de Python para importar epsilon
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Verificación visual
print(f"Buscando módulos en: {current_dir}")

# Intentar importar la biblioteca epsilon
try:
    import epsilon
    print("✅ Biblioteca epsilon cargada correctamente")
except SyntaxError as e:
    print("❌ ERROR DE SINTAXIS al importar epsilon.py")
    print(f"   {e}")
    print("\n   Esto suele ocurrir cuando epsilon.py no es un archivo Python válido.")
    print("   Verifica que epsilon.py sea código Python, no un notebook JSON.")
    raise
except ImportError as e:
    print("❌ ERROR DE IMPORTACIÓN")
    print(f"   {e}")
    print("\n   Verifica que epsilon.py esté en el directorio actual")
    raise
except Exception as e:
    print(f"❌ ERROR INESPERADO: {type(e).__name__}")
    print(f"   {e}")
    raise

# Habilitar widgets interactivos
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

print("✅ Entorno configurado correctamente")

Buscando módulos en: /mnt/scratch/HySEALab


NameError: name 'null' is not defined

### Paso 2: Estructura de directorios

Por defecto, epsilon busca las simulaciones en la carpeta `./simulaciones/`. Asegúrate de tener la siguiente estructura:

```
tu_proyecto/
├── epsilon.py
├── Manual_Usuario_Epsilon.ipynb  (este notebook)
├── simulaciones/
│   ├── archivo1.nc
│   ├── archivo2.nc
│   └── Carpeta_Simulacion/
│       ├── subarchivo1.nc
│       └── subarchivo2.nc
└── plots/  (se creará automáticamente para guardar figuras)
```

### Paso 3: Iniciar la interfaz

La función principal para comenzar es `display()`. Esta mostrará todos los archivos .nc y carpetas disponibles en `./simulaciones/`

In [ ]:
# Iniciar la interfaz de selección de simulaciones
epsilon.display()

**¿Qué hace este comando?**

- Escanea el directorio `./simulaciones/`
- Crea botones para cada archivo .nc encontrado
- Crea botones para carpetas (marcados con `[F]`)
- Al hacer clic en un botón, carga la simulación y muestra el mapa base

---
## 4. Funcionalidades Principales <a name="funcionalidades"></a>

### 4.1 Selección de Simulaciones <a name="seleccion"></a>

#### Opción A: Archivo individual

Si tienes un único archivo NetCDF con toda la simulación:

1. Ejecuta `epsilon.display()`
2. Haz clic en el botón del archivo deseado (ej: "japon7 Output")
3. Se mostrará el mapa de batimetría/topografía
4. Aparecerá un menú para seleccionar variables

#### Opción B: Carpeta con múltiples archivos

Si tu simulación está dividida en varios archivos (ej: por regiones o mallas anidadas):

1. Ejecuta `epsilon.display()`
2. Haz clic en el botón de carpeta (marcado con `[F]`)
3. Se mostrará un mapa con rectángulos indicando las diferentes regiones
4. Haz clic en el botón del archivo que deseas analizar

#### Ejemplo de uso:

In [ ]:
# Mostrar simulaciones disponibles
epsilon.display()

# Después de ejecutar esta celda, verás botones interactivos
# Haz clic en el que corresponda a tu simulación

#### Especificar un directorio personalizado:

Si tus simulaciones están en otro directorio:

In [ ]:
# Usar un directorio personalizado
epsilon.display(path="/ruta/a/tus/simulaciones")

---
### 4.2 Visualización de Variables 2D <a name="visual-2d"></a>

Las variables 2D son aquellas que no dependen del tiempo, como:

- **original_bathy** / **bathymetry**: Batimetría/topografía original
- **deformed_bathy**: Batimetría deformada por el terremoto
- **arrival_times**: Tiempos de llegada del tsunami
- **max_height**: Altura máxima de ola

#### Flujo de trabajo:

1. Selecciona una simulación
2. Aparecerá un menú con todas las variables disponibles
3. Haz clic en la variable que deseas visualizar
4. Se mostrará el mapa con un slider para ajustar la escala de colores

#### Características de la visualización 2D:

- **Slider de escala**: Ajusta los valores mínimo y máximo de la escala de colores
- **Línea de costa**: Contorno negro en el nivel 0 (tierra/mar)
- **Botón "Save figure"**: Guarda la imagen actual en `plots/mapa_variable_2d.png`

#### Opciones adicionales:

Después de visualizar una variable 2D, aparece el botón:

- **"Make transect"**: Crea un perfil transversal de la variable (ver sección 4.4)

#### Ejemplo paso a paso:

In [ ]:
# 1. Cargar simulación
epsilon.display()

# 2. Hacer clic en tu simulación (ej: "japon7 Output")
# 3. Hacer clic en una variable 2D (ej: "original_bathy")
# 4. Ajustar el slider de escala según necesites
# 5. Clic en "Save figure" si deseas guardar la imagen

---
### 4.3 Visualización de Variables 3D (Temporales) <a name="visual-3d"></a>

Las variables 3D incluyen la dimensión temporal y permiten ver la evolución de la variable a lo largo del tiempo:

- **eta**: Altura de la superficie del agua
- **u**: Velocidad en dirección X
- **v**: Velocidad en dirección Y
- **qx**, **qy**: Flujo de momento

#### Flujo de trabajo:

1. Selecciona una simulación
2. Haz clic en una variable 3D (ej: "eta")
3. Se mostrará el mapa con dos sliders:
   - **Time Step**: Selecciona el instante temporal
   - **Scale**: Ajusta la escala de colores

#### Características de la visualización 3D:

- **Slider temporal**: Navega entre diferentes instantes de tiempo
- **Título dinámico**: Muestra el tiempo en segundos del paso actual
- **Botón "Save Figure"**: Guarda la imagen del instante actual
- **Botón "Save animated GIF"**: Crea una animación con todos los pasos temporales

#### Opciones adicionales:

Después de visualizar una variable 3D, aparecen los botones:

- **"Extract time series"**: Extrae la evolución temporal en un punto (ver sección 4.5)
- **"Make a transect"**: Crea un perfil transversal en un instante específico (ver sección 4.4)

#### Generación de GIF animado:

El botón **"Save animated GIF"** realiza las siguientes acciones:

1. Genera imágenes individuales para cada paso temporal
2. Las guarda temporalmente en `frames_gif/`
3. Combina todas las imágenes en `plots/animacion_variable.gif`
4. Elimina los archivos temporales

⚠️ **Nota**: La generación de GIF puede tardar varios minutos dependiendo del número de pasos temporales.

#### Ejemplo:

In [ ]:
# 1. Cargar simulación
epsilon.display()

# 2. Hacer clic en tu simulación
# 3. Hacer clic en una variable temporal (ej: "eta")
# 4. Usar el slider "Time Step" para navegar en el tiempo
# 5. Ajustar la escala de colores con el slider "Scale"
# 6. Opcional: Clic en "Save animated GIF" para crear una animación

---
### 4.4 Perfiles Transversales <a name="transversales"></a>

Los perfiles transversales permiten ver cómo varían las variables a lo largo de una línea recta entre dos puntos.

#### Para variables 2D:

1. Visualiza una variable 2D
2. Haz clic en **"Make transect"**
3. Aparecerá una cuadrícula numerada (5×5 = 25 celdas)
4. **Opción A**: Haz clic en dos celdas para definir el inicio y fin del transecto
5. **Opción B**: Introduce coordenadas manualmente (X1, Y1, X2, Y2) y haz clic en "Continuar"
6. Se mostrará:
   - Mapa con línea negra del corte
   - Gráfico del perfil con los valores de la variable

#### Para variables 3D:

1. Visualiza una variable 3D
2. Haz clic en **"Make a transect"**
3. Usa el slider para seleccionar el instante temporal deseado
4. Haz clic en **"grabar tiempo"** para fijar ese instante
5. Sigue los mismos pasos que para variables 2D

#### Interpretación del perfil:

- **Eje X**: Distancia a lo largo del corte (en índices de array o distancia real)
- **Eje Y**: Valor de la variable seleccionada
- La línea azul muestra cómo varía la variable entre los dos puntos

#### Ejemplo de uso:

In [ ]:
# Caso de uso: Perfil batimétrico perpendicular a la costa

# 1. Cargar simulación y seleccionar "original_bathy"
epsilon.display()

# 2. Clic en "Make transect"
# 3. Seleccionar dos celdas: una en tierra y otra en mar profundo
#    Por ejemplo: celda 3 (tierra) y celda 23 (mar)
# 4. Verás el perfil batimétrico mostrando la pendiente del fondo marino

---
### 4.5 Series Temporales <a name="series-temporales"></a>

Las series temporales muestran cómo evoluciona una variable en un punto específico a lo largo del tiempo. Es especialmente útil para:

- Simular maregrafos virtuales
- Analizar la llegada del tsunami a un punto
- Estudiar la amplitud y frecuencia de las olas

#### Flujo de trabajo:

1. Visualiza una variable 3D (ej: "eta")
2. Haz clic en **"Extract time series"**
3. Aparecerá:
   - Un slider temporal para explorar visualmente la malla
   - Una cuadrícula numerada (5×5)
   - Campos para introducir coordenadas manualmente
4. **Opción A**: Haz clic en una celda
5. **Opción B**: Introduce coordenadas (X, Y) y haz clic en "Continuar"
6. Se mostrará un gráfico con:
   - **Eje X**: Tiempo (segundos)
   - **Eje Y**: Valor de la variable

#### Interpretación del gráfico:

Para una variable "eta" (altura del agua):

- **Valor inicial ≈ 0**: Nivel del mar en reposo
- **Primera oscilación**: Llegada de la primera ola
- **Amplitud de picos**: Altura máxima de las olas
- **Periodo entre picos**: Periodo de las olas del tsunami
- **Decaimiento**: Atenuación del tsunami con el tiempo

#### Caso especial: Boyas

Si el archivo NetCDF representa datos de una boya puntual (1×1 espacialmente), epsilon detectará automáticamente esto y mostrará directamente la serie temporal sin necesidad de seleccionar una celda.

#### Ejemplo:

In [ ]:
# Caso de uso: Mareógrafo virtual en un puerto

# 1. Cargar simulación y seleccionar "eta"
epsilon.display()

# 2. Clic en "Extract time series"
# 3. Introducir las coordenadas del puerto:
#    Por ejemplo: X = -6.35, Y = 36.53 (Cádiz)
# 4. Clic en "Continuar"
# 5. Analizar el gráfico resultante:
#    - ¿Cuándo llega la primera ola?
#    - ¿Cuál es la altura máxima?
#    - ¿Cuántas olas se observan?

---
### 4.6 Análisis de Inundación <a name="inundacion"></a>

El análisis de inundación calcula y visualiza las zonas terrestres que son inundadas por el tsunami.

#### Requisitos:

El archivo NetCDF debe contener:
- **eta**: Altura de la superficie del agua
- **original_bathy** o **bathymetry**: Batimetría/topografía

Si ambas variables están presentes, aparecerá automáticamente el botón **"flood"** en el menú de variables.

#### Cálculo de inundación:

La biblioteca realiza los siguientes cálculos:

1. **Máscara de tierra**: `mask = bathy < 0` (zonas con topografía negativa = mar)
2. **Profundidad de inundación**: `inundacion = eta - bathy` (en zonas terrestres)
3. **Filtrado**: Se eliminan valores negativos (zonas no inundadas)

#### Visualización:

- **Escala de colores**: De -1 a 2 metros (configurable)
- **Contorno negro**: Línea de costa (bathy = 0)
- **Slider temporal**: Navega por la evolución de la inundación
- **Valores NaN**: Las zonas no inundadas se muestran transparentes

#### Botones disponibles:

- **"Guardar imagen"**: Guarda el instante actual
- **"Guardar GIF"**: Crea animación de toda la secuencia de inundación

#### Interpretación de resultados:

- **Colores azules/fríos**: Poca profundidad de inundación (< 0.5 m)
- **Colores cálidos/rojos**: Mayor profundidad de inundación (> 1 m)
- **Áreas extensas inundadas**: Indican zonas de bajo relieve vulnerable

#### Limitaciones:

⚠️ Por razones de rendimiento, si el dataset es muy grande (> 200 millones de datos), epsilon recortará automáticamente el número de pasos temporales procesados.

#### Ejemplo:

In [ ]:
# Análisis de inundación

# 1. Cargar simulación
epsilon.display()

# 2. En el menú de variables, buscar el botón "flood"
# 3. Hacer clic en "flood"
# 4. Usar el slider temporal para ver la evolución de la inundación
# 5. Identificar:
#    - Zonas de máxima inundación
#    - Tiempo en que ocurre la inundación máxima
#    - Extensión espacial del área inundada
# 6. Guardar GIF para documentación o presentaciones

---
### 4.7 Tiempos de Llegada del Tsunami <a name="arrival-times"></a>

Los mapas de tiempos de llegada (arrival times) muestran cuánto tiempo tarda en llegar el tsunami a cada punto de la costa.

#### Variable: `arrival_times`

- **Valores**: Tiempo en segundos (convertido a minutos en la visualización)
- **Valor especial -1**: Indica que el tsunami no llega a ese punto

#### Visualización especializada:

Epsilon detecta automáticamente la variable `arrival_times` y aplica una visualización optimizada:

1. **Conversión automática**: Segundos → Minutos
2. **Mapa de colores**: "coolwarm" (azul = llegada rápida, rojo = llegada tardía)
3. **Isolíneas**: Curvas negras cada 5 minutos con etiquetas
4. **Filtrado**: Zonas con valor -1 se muestran como NaN (transparente)

#### Slider de escala:

Permite ajustar el rango de tiempos visualizados:
- **Útil para**: Enfocarse en una ventana temporal específica
- **Ejemplo**: Mostrar solo los primeros 30 minutos

#### Interpretación del mapa:

- **Isolíneas cerradas**: Puntos de la costa con el mismo tiempo de llegada
- **Gradiente de color**: Indica la velocidad de propagación
- **Zonas sin dato**: Mar abierto o áreas donde la ola es muy pequeña

#### Aplicaciones:

- **Sistemas de alerta temprana**: Identificar zonas con menos tiempo de evacuación
- **Planificación de evacuación**: Diseñar rutas según tiempo disponible
- **Validación de modelos**: Comparar con datos históricos de tsunamis

#### Ejemplo:

In [ ]:
# Mapa de tiempos de llegada

# 1. Cargar simulación que contenga "arrival_times"
epsilon.display()

# 2. Hacer clic en "arrival_times"
# 3. Se mostrará el mapa con isolíneas cada 5 minutos
# 4. Interpretar:
#    - ¿Qué punto de la costa se inunda primero?
#    - ¿Cuánto tiempo hay para evacuar desde el terremoto?
#    - ¿Qué zonas tienen más de 30 min de margen?
# 5. Ajustar slider si necesitas enfocarte en un rango específico
# 6. Guardar figura para informes

---
### 4.8 Operaciones entre Variables <a name="operaciones"></a>

Epsilon permite realizar operaciones algebraicas entre dos variables del mismo archivo NetCDF.

#### Operaciones disponibles:

- **Suma** (+): `variable1 + variable2`
- **Resta** (−): `variable1 - variable2`
- **Multiplicación** (×): `variable1 * variable2`

#### Casos de uso comunes:

1. **Deformación cosísmica**: `deformed_bathy - original_bathy`
2. **Altura de ola total**: `eta + original_bathy` (profundidad desde el fondo)
3. **Velocidad total**: `sqrt(u² + v²)` → requiere calcular `u*u`, `v*v` y sumar
4. **Energía cinética**: `0.5 * (qx*u + qy*v)`

#### Flujo de trabajo:

1. En el menú de variables, haz clic en **"Operate variables"**
2. Aparecerán toggles (botones on/off) para cada variable
3. Activa **dos variables** (el sistema no permite más de dos)
4. Cuando selecciones la segunda, aparecerán botones: `+`, `-`, `*`
5. Haz clic en la operación deseada
6. El resultado se visualizará automáticamente

#### Compatibilidad dimensional:

Epsilon maneja automáticamente las diferencias dimensionales:

- **2D + 2D**: Resultado 2D
- **3D + 3D**: Resultado 3D (operación en cada paso temporal)
- **2D + 3D**: La variable 2D se expande automáticamente para operar con cada paso temporal de la 3D

⚠️ **Limitación**: Para datasets muy grandes (> 200M datos), epsilon recorta automáticamente los pasos temporales.

#### Visualización del resultado:

- El resultado se trata como una variable nueva
- Si es 2D: Se muestra con `mostmap2d()`
- Si es 3D: Se muestra con `mostmap3d()` (con sliders)
- El título indica la operación realizada

#### Ejemplo práctico:

In [ ]:
# Ejemplo 1: Calcular la deformación cosísmica

# 1. Cargar simulación
epsilon.display()

# 2. Clic en "Operate variables"
# 3. Activar toggle de "deformed_bathy"
# 4. Activar toggle de "original_bathy"
# 5. Clic en el botón "-" (resta)
# 6. El resultado muestra el levantamiento/subsidencia del fondo marino
#    - Valores positivos: Levantamiento
#    - Valores negativos: Subsidencia

# Ejemplo 2: Magnitud de la velocidad
# Para calcular sqrt(u² + v²) debes hacer:
# Paso 1: u * u → guarda mentalmente el resultado
# Paso 2: v * v → guarda mentalmente el resultado  
# Paso 3: Rehacer la operación "Operate variables" para sumar u² + v²
# (Actualmente no se puede guardar resultados intermedios, es una limitación)

---
## 5. Exportación de Resultados <a name="exportacion"></a>

### 5.1 Guardar imágenes estáticas

**Ubicación**: Todas las figuras se guardan en el directorio `plots/`

**Variables 2D**:
- Botón: **"Save figure"**
- Archivo: `plots/mapa_variable_2d.png`

**Variables 3D**:
- Botón: **"Save Figure"**
- Archivo: `plots/mapa_variable_3d.png`
- Nota: Se guarda el instante temporal mostrado en el slider

### 5.2 Generar animaciones GIF

**Disponible para**: Variables 3D y análisis de inundación

**Botón**: **"Save animated GIF"** o **"Guardar GIF"**

**Archivo**: `plots/animacion_variable.gif`

**Proceso**:
1. Se crea una carpeta temporal `frames_gif/`
2. Se genera una imagen PNG por cada paso temporal
3. Se combinan todas en un GIF animado
4. Se eliminan los archivos temporales
5. El GIF final queda en `plots/`

**Duración del GIF**:
- Variables normales: 0.01 segundos por frame (muy rápido)
- Inundación: 6 segundos por frame (más lento para análisis)

⚠️ **Advertencias**:
- Los GIF pueden ser archivos muy grandes (> 100 MB) si hay muchos pasos temporales
- El proceso puede tardar varios minutos
- No cierres el navegador durante la generación

### 5.3 Verificar archivos guardados

In [ ]:
# Listar archivos guardados en el directorio plots/
import os

if os.path.exists('plots'):
    archivos = os.listdir('plots')
    if archivos:
        print("📁 Archivos en plots/:")
        for archivo in sorted(archivos):
            ruta = os.path.join('plots', archivo)
            tamaño = os.path.getsize(ruta) / (1024*1024)  # MB
            print(f"  - {archivo} ({tamaño:.2f} MB)")
    else:
        print("⚠️ El directorio plots/ está vacío")
else:
    print("⚠️ El directorio plots/ no existe aún")
    print("💡 Se creará automáticamente al guardar la primera figura")

### 5.4 Personalizar ubicación de guardado

Si deseas cambiar el directorio donde se guardan las figuras, deberás modificar el archivo `epsilon.py`. Busca las líneas que contienen:

```python
fig.savefig("plots/nombre_archivo.png")
```

Y cámbialo por tu ruta preferida:

```python
fig.savefig("/mi/ruta/personalizada/nombre_archivo.png")
```

---
## 6. Ejemplos Prácticos <a name="ejemplos"></a>

### Ejemplo 1: Análisis completo de un tsunami histórico

**Objetivo**: Analizar la simulación del tsunami de Japón 2011

**Pasos**:

In [ ]:
# Paso 1: Cargar simulación de Japón
epsilon.display()
# → Clic en "japon7 Output"

# Paso 2: Visualizar batimetría deformada
# → Clic en "deformed_bathy"
# → Observar la zona de deformación cosísmica

# Paso 3: Calcular deformación
# → Volver atrás y clic en "Operate variables"
# → Activar "deformed_bathy" y "original_bathy"
# → Clic en "-" (resta)
# → Guardar figura

# Paso 4: Analizar propagación del tsunami
# → Volver y seleccionar "eta"
# → Usar slider temporal para ver evolución
# → Guardar GIF animado

# Paso 5: Tiempos de llegada
# → Seleccionar "arrival_times"
# → Identificar zonas de evacuación prioritaria
# → Guardar figura

# Paso 6: Mareógrafo virtual en Sendai
# → Volver a "eta"
# → Clic en "Extract time series"
# → Introducir coordenadas de Sendai: X=141.0, Y=38.3
# → Analizar el mareograma

### Ejemplo 2: Análisis de inundación en zona costera

**Objetivo**: Determinar áreas inundadas y profundidad máxima

In [ ]:
# Paso 1: Cargar simulación de zona costera
epsilon.display()
# → Clic en la carpeta "Cadiz Output" (ejemplo)

# Paso 2: Seleccionar archivo de detalle costero
# → Clic en el botón del archivo con mayor resolución

# Paso 3: Análisis de inundación
# → Clic en "flood"
# → Usar slider para encontrar el instante de máxima inundación
# → Observar:
#    * Profundidad máxima (valores en escala de colores)
#    * Extensión horizontal
#    * Zonas más vulnerables

# Paso 4: Exportar resultados
# → Posicionar slider en el máximo
# → Guardar imagen
# → Guardar GIF de toda la secuencia

# Paso 5: Análisis cuantitativo (avanzado)
# → Puedes extraer los datos manualmente del NetCDF para análisis estadístico

### Ejemplo 3: Comparación de escenarios

**Objetivo**: Comparar dos escenarios de tsunami con diferente magnitud

In [ ]:
# Suponiendo que tienes dos simulaciones: escenario_M8.nc y escenario_M9.nc

# ESCENARIO 1: Magnitud 8.0
epsilon.display()
# → Clic en "escenario_M8 Output"
# → Seleccionar "arrival_times" → Guardar como plots/M8_arrival.png
# → Seleccionar "eta" → Guardar GIF como plots/M8_propagacion.gif
# → Extraer serie temporal en punto crítico → Anotar altura máxima

# ESCENARIO 2: Magnitud 9.0
# (Repetir proceso ejecutando epsilon.display() nuevamente)
# → Clic en "escenario_M9 Output"
# → Mismos análisis que arriba

# COMPARACIÓN:
# → Abrir ambas imágenes de arrival_times
# → ¿Cuánto disminuye el tiempo de llegada en M9 vs M8?
# → Comparar alturas máximas en el punto crítico
# → Estimar el aumento del riesgo

---
## 7. Solución de Problemas <a name="problemas"></a>

### Problema 1: No aparecen botones después de ejecutar `epsilon.display()`

**Posibles causas**:
- Los widgets no están habilitados en Jupyter
- El directorio `./simulaciones/` no existe o está vacío

**Solución**:

In [ ]:
# Verificar extensión de widgets
# En terminal:
# jupyter nbextension enable --py widgetsnbextension

# Verificar directorio
import os
if os.path.exists('./simulaciones'):
    archivos = os.listdir('./simulaciones')
    print(f"✓ Directorio existe. Archivos: {len(archivos)}")
    if not archivos:
        print("⚠️ Directorio vacío. Añade archivos .nc")
else:
    print("✗ Directorio './simulaciones' no existe")
    os.makedirs('./simulaciones')
    print("✓ Directorio creado. Añade archivos .nc ahí")

### Problema 3: Error al cargar archivo NetCDF

**Error típico**: `FileNotFoundError` o `OSError`

**Solución**:

### Problema 1b: Error GMTCLibNotFoundError con PyGMT

**Error típico**: `GMTCLibNotFoundError: Error loading GMT shared library at 'libgmt.so'`

**Causa**: PyGMT está instalado pero GMT (Generic Mapping Tools) no está instalado en el sistema operativo.

**Soluciones**:

In [ ]:
# Solución 1: Instalar GMT con conda (RECOMENDADO)
# En terminal:
# conda install -c conda-forge gmt
# pip install pygmt

# Solución 2: Instalar GMT con homebrew (macOS)
# En terminal:
# brew install gmt
# pip install pygmt

# Solución 3: Usar epsilon.py sin PyGMT
# epsilon.py funcionará correctamente sin PyGMT
# Solo no mostrará los mapas base iniciales con PyGMT
# Las visualizaciones con matplotlib funcionarán normalmente

# Verificar si GMT está disponible:
import subprocess
import os

print("Verificando instalación de GMT:\n")

# Método 1: Verificar con comando gmt
try:
    result = subprocess.run(['gmt', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ GMT instalado: versión {result.stdout.strip()}")
        print("   → Ahora puedes instalar PyGMT: pip install pygmt")
    else:
        print("❌ GMT no encontrado en el sistema")
except FileNotFoundError:
    print("❌ GMT no está instalado en el PATH del sistema")
    print("\nOpciones de instalación:")
    print("  1. conda install -c conda-forge gmt  (recomendado)")
    print("  2. brew install gmt  (macOS)")
    print("  3. sudo apt-get install gmt gmt-dcw gmt-gshhg  (Linux)")

# Método 2: Verificar variables de entorno
gmt_home = os.environ.get('GMT_HOME')
if gmt_home:
    print(f"\n✓ GMT_HOME configurado: {gmt_home}")
else:
    print("\n⚠️ Variable GMT_HOME no configurada")

print("\n💡 NOTA: epsilon.py funciona sin PyGMT")
print("   Solo afecta a los mapas base iniciales")
print("   Todas las demás funcionalidades (matplotlib) funcionan normalmente")

In [ ]:
# Verificar que el archivo existe y es accesible
import netCDF4 as nc

archivo = "./simulaciones/tu_archivo.nc"  # Ajustar ruta

try:
    dataset = nc.Dataset(archivo, mode="r")
    print(f"✓ Archivo cargado correctamente")
    print(f"Variables disponibles: {list(dataset.variables.keys())}")
    dataset.close()
except FileNotFoundError:
    print(f"✗ Archivo no encontrado: {archivo}")
    print("Verifica la ruta y el nombre del archivo")
except OSError as e:
    print(f"✗ Error al leer el archivo: {e}")
    print("El archivo puede estar corrupto o no ser un NetCDF válido")

### Problema 4: Visualización muy lenta o memoria insuficiente

**Causa**: Datasets muy grandes (alta resolución o muchos pasos temporales)

**Soluciones automáticas** (ya implementadas en epsilon):
- Compresión automática si el array tiene > 4M elementos
- Recorte de pasos temporales si > 200M datos totales

**Si sigue lento**:
1. Cierra otras pestañas/programas
2. Reinicia el kernel de Jupyter
3. Trabaja con un subset del dominio espacial

### Problema 5: Coordenadas incorrectas (mapa distorsionado)

**Causa**: Coordenadas UTM no convertidas correctamente

**Solución**: Verificar zona UTM en el código

In [ ]:
# La conversión UTM está hardcoded en epsilon.py
# Busca la línea:
# utm_zone = 30

# Ajusta según tu zona:
# - España: Zona 29, 30, 31 (según longitud)
# - Japón: Zonas 51-56
# - Chile: Zonas 18-19
# - etc.

# Calculadora de zona UTM:
def calcular_zona_utm(longitud):
    zona = int((longitud + 180) / 6) + 1
    return zona

# Ejemplo:
print(f"Zona UTM para longitud -6.3 (Cádiz): {calcular_zona_utm(-6.3)}")
print(f"Zona UTM para longitud 141.0 (Sendai): {calcular_zona_utm(141.0)}")

### Problema 6: GIF no se genera o queda incompleto

**Posibles causas**:
- Proceso interrumpido (navegador cerrado)
- Espacio en disco insuficiente
- Demasiados frames

**Solución**:

In [ ]:
# Verificar espacio en disco
import shutil

total, usado, libre = shutil.disk_usage(".")
print(f"Espacio total: {total // (2**30)} GB")
print(f"Espacio usado: {usado // (2**30)} GB")
print(f"Espacio libre: {libre // (2**30)} GB")

if libre < 1e9:  # Menos de 1 GB
    print("⚠️ Poco espacio libre. Libera espacio antes de generar GIFs")

# Limpiar carpeta temporal si existe
import os
if os.path.exists('frames_gif'):
    shutil.rmtree('frames_gif')
    print("✓ Carpeta temporal eliminada")

### Problema 7: Variables no aparecen en el menú

**Causa**: La variable puede ser 1D o escalar

**Verificación**:

In [ ]:
# Inspeccionar estructura del NetCDF
import netCDF4 as nc

archivo = "./simulaciones/tu_archivo.nc"  # Ajustar
dataset = nc.Dataset(archivo, mode="r")

print("Variables y dimensiones:\n")
for var in dataset.variables:
    dims = dataset.variables[var].shape
    tipo = "1D" if len(dims) == 1 else f"{len(dims)}D"
    visible = "✓ Visible" if len(dims) > 1 else "✗ Oculta"
    print(f"{var:20s} {tipo:5s} {str(dims):20s} {visible}")

dataset.close()

# Nota: Epsilon solo muestra variables con 2 o más dimensiones

---
## 8. Referencias <a name="referencias"></a>

### Documentación relacionada:

- **Tsunami-HySEA**: [https://edanya.uma.es/hysea/](https://edanya.uma.es/hysea/)
- **NetCDF**: [https://www.unidata.ucar.edu/software/netcdf/](https://www.unidata.ucar.edu/software/netcdf/)
- **PyGMT**: [https://www.pygmt.org/](https://www.pygmt.org/)
- **Matplotlib**: [https://matplotlib.org/](https://matplotlib.org/)
- **ipywidgets**: [https://ipywidgets.readthedocs.io/](https://ipywidgets.readthedocs.io/)

### Publicaciones científicas:

Para citar Tsunami-HySEA en publicaciones, consulta:

- Macías, J. et al. (2017). "Performance benchmarking of tsunami-HySEA model for NTHMP's inundation mapping activities". *Pure and Applied Geophysics*, 174(8), 3147-3183.

### Contacto y soporte:

- **Email**: hysea@uma.es
- **Web**: [https://edanya.uma.es/](https://edanya.uma.es/)
- **Institución**: HySEALab - Universidad de Málaga

### Licencia:

Este manual y la biblioteca epsilon.py son desarrollados por HySEALab para uso académico y de investigación.

---

**Última actualización**: Febrero 2026  
**Versión del manual**: 1.0  
**Compatibilidad**: epsilon.py v1.0

---
## Apéndice A: Estructura de un archivo NetCDF típico de Tsunami-HySEA

### Variables espaciales (2D):
```
lon(x)              - Longitud [grados o metros]
lat(y)              - Latitud [grados o metros]
original_bathy(y,x) - Batimetría original [m]
deformed_bathy(y,x) - Batimetría deformada [m]
arrival_times(y,x)  - Tiempos de llegada [s]
```

### Variables temporales (3D):
```
time(t)        - Tiempo [s]
eta(t,y,x)     - Altura superficie libre [m]
u(t,y,x)       - Velocidad en X [m/s]
v(t,y,x)       - Velocidad en Y [m/s]
qx(t,y,x)      - Flujo de momento X [m²/s]
qy(t,y,x)      - Flujo de momento Y [m²/s]
```

### Atributos globales típicos:
```
title          - Nombre de la simulación
institution    - Universidad de Málaga
source         - Tsunami-HySEA
references     - Publicaciones
```

---
## Apéndice B: Flujo de trabajo completo (diagrama)

```
┌─────────────────────────┐
│  epsilon.display()      │
└───────────┬─────────────┘
            │
            ├─→ Archivo único → elegirdeuna()
            │                      │
            │                      └─→ elegirop() → menú variables
            │                                          │
            │                                          ├─→ Variable 2D → menu2d()
            │                                          │       │
            │                                          │       ├─→ Transecto
            │                                          │       └─→ Guardar
            │                                          │
            │                                          ├─→ Variable 3D → menu3d()
            │                                          │       │
            │                                          │       ├─→ Serie temporal
            │                                          │       ├─→ Transecto
            │                                          │       └─→ Guardar/GIF
            │                                          │
            │                                          ├─→ flood → menu_inun()
            │                                          │
            │                                          └─→ Operar → mostrar resultado
            │
            └─→ Carpeta → elegirarch()
                             │
                             └─→ multiarch() → botones por archivo
                                                  │
                                                  └─→ elegirdemult() → (igual que archivo único)
```

---
## Apéndice C: Atajos y consejos

### Atajos de teclado en Jupyter:

- **Shift + Enter**: Ejecutar celda y avanzar
- **Ctrl + Enter**: Ejecutar celda y quedarse
- **Alt + Enter**: Ejecutar celda e insertar nueva debajo
- **Esc**: Entrar en modo comando
- **A** (en modo comando): Insertar celda arriba
- **B** (en modo comando): Insertar celda abajo
- **DD** (en modo comando): Eliminar celda

### Consejos de uso:

1. **Trabaja con copias**: Siempre ten respaldos de tus archivos .nc originales
2. **Nombrado descriptivo**: Renombra tus simulaciones con nombres claros
3. **Organización**: Usa subcarpetas en `simulaciones/` para diferentes proyectos
4. **Rendimiento**: Para datasets grandes, trabaja primero con un subset temporal
5. **Documentación**: Añade celdas markdown en tus notebooks para documentar hallazgos
6. **Exportación**: Guarda figuras regularmente, no esperes al final
7. **Verificación**: Siempre verifica que las coordenadas son correctas antes de análisis
8. **Series temporales**: Extrae en múltiples puntos para comparación
9. **GIFs**: Ajusta la escala de colores antes de generar el GIF para consistencia
10. **Limpieza**: Borra archivos temporales y GIFs antiguos periódicamente

### Flujo de trabajo recomendado:

1. Inspeccionar batimetría/topografía
2. Analizar deformación cosísmica (si aplica)
3. Ver propagación general (eta, animación rápida)
4. Mapas de tiempos de llegada
5. Análisis detallado de inundación (si hay zonas costeras)
6. Series temporales en puntos críticos
7. Perfiles transversales en zonas de interés
8. Exportación final de todas las figuras
9. Documentación de resultados en el notebook

---

**¡Fin del manual! ¡Buena suerte con tus análisis de tsunamis!** 🌊